In [18]:
with open('/Users/mko_san/Downloads/py2.txt', 'r') as py2, open('/Users/mko_san/Downloads/py3.txt', 'r') as py3:
    lines = [line.replace('\n', '') for line in py3 if '.' not in line]
    lines += [line.replace('\n', '') for line in py2 if '.' not in line]
    lines = list(set(lines))

from stdlib_list import stdlib_list
libraries = stdlib_list("2.7") + stdlib_list("3.6")
lines = list(set(libraries))
print (lines)

In [172]:
import requests

url = 'https://crossword.stxnext.com/api/pyconpl2017/challenges/365'
cookie = {'session': '.eJwlj0lqAzEQAP-isw-tXqSWPzNMb8QYEpixTyF_90DORUHVb9vqyPOr3V_HO29te0S7t1FiRjAFigMwxz6B9hjpZnkBscBCVmBfQlJCNPYC9MEGq0om7ivWgBxMac7Wg4jBTSBZfbiGghHbpLCe2QEgFKWvmVnebs3Po7bXzzO_r55Swd2C5nKZKyYTc2eokbUwUBXEEXVd3vvM43-Ctf19AAZ0PuY.DHh1Rg.tEB3jT39VrjOlBSf91s_pxlxIcI'}
r = requests.get(url, cookies=cookie)

In [65]:
from scrapy.selector import Selector
url_base = 'https://crossword.stxnext.com/pyconpl2017/challenges/368'
req = requests.get(url_base, cookies=cookie)
hints = Selector(text=req.content.decode()).xpath('/html/body/section/div/div/div[1]/pre/code/text()').extract()[0].split('\n')

In [255]:
data = r.content.decode().split("\n")
print(*data, sep='\n')

               c  r 
              s**ket
     d         l    
     a       s *    
     * t     u e    
     e h***lib *sv  
     * r y   p t    
t*mp**l* s w***ings 
  a  m a     o o    
  *  e *  j  c n    
  h    * o*  e s    
       n  o  s      
     l***ing s      


In [256]:
def get_words(data, down=True):
    words = []
    word = ''
    rows = len(data)
    columns = len(data[0])
    

    if down:
        for i in range(0, columns):
            for j in range(0, rows):
                if data[j][i] != ' ':
                    word += data[j][i]
                elif data[j][i] == ' ':
                    if word:
                        if len(word) > 1:
                            words.append((word, (j-1, i), down))
                    word = ''
            if word:
                if len(word) > 1:
                    words.append((word, (j, i), down))
                word = ''
    else:
        for j in range(0, rows):
            for i in range(0, columns):
                if data[j][i] != ' ':
                    word += data[j][i]
                elif data[j][i] == ' ':
                    if word:
                        if len(word) > 1:
                            words.append((word, (j, i-1), down))
                    word = ''
            if word:
                if len(word) > 1:
                    words.append((word, (j, i), down))
                word = ''
    return words

words = get_words(data, False)
words += get_words(data, True)
all_words = [a[0] for a in words]
all_words         

['s**ket',
 'h***lib',
 '*sv',
 't*mp**l*',
 'w***ings',
 'o*',
 'l***ing',
 'ma*h',
 'da*e**me',
 'thr*a**n*',
 '*ys',
 'j*on',
 'subp*ocess',
 'c*l*e*tions',
 're']

In [257]:
import re
def find_matches(words):
    result = []
    words.sort(key = lambda s: len(s[0]))
    words.reverse()
    for word in words:
        matches = []
        for line in lines:
            if len(line) == len(word[0]):
                match = re.match(word[0].replace('*', '.'), line)
                if match:
                    matches.append(match[0])
        word = (word[0], word[1], word[2], tuple(matches))
        print (word)
        result.append(word)
    return result
    
words = find_matches(words)

('c*l*e*tions', (10, 15), True, ('collections',))
('subp*ocess', (12, 13), True, ('subprocess',))
('thr*a**n*', (12, 7), True, ('threading',))
('da*e**me', (9, 5), True, ('datetime',))
('w***ings', (7, 18), False, ('warnings',))
('t*mp**l*', (7, 7), False, ('tempfile',))
('l***ing', (12, 11), False, ('logging',))
('h***lib', (5, 13), False, ('htmllib', 'hashlib', 'httplib'))
('s**ket', (1, 19), False, ('socket',))
('j*on', (12, 10), True, ('json',))
('ma*h', (10, 2), True, ('math',))
('*ys', (7, 9), True, ('sys',))
('*sv', (5, 17), False, ('csv',))
('re', (1, 18), True, ('re',))
('o*', (10, 10), False, ('os',))


In [258]:
def data_fill(data, word):
    j, i = word[1]
    down = word[2]
    match = word[3][0]
    length = len(match)
    
    if down:
        j -= length - 1
        x = 0
        for j in range(j, j+length):
            data[j] = list(data[j])
            data[j][i] = match[x]
            data[j] = ''.join(data[j])
            x += 1
    else:
        i -= length - 1
        x = 0
        
        data[j] = list(data[j])
        for i in range(i, i+length):
            data[j][i] = match[x]
            x += 1
        data[j] = ''.join(data[j])
    return data

def solve(data, words, rerun=False):
    to_try = []
    for word in words:
        if len(word[3]) == 1:
            data = data_fill(data, word)
    words = get_words(data, False) +  get_words(data, True)
    words = find_matches(words)
    if not rerun:
        data = solve(data, words, True)
    print(*data, sep='\n')
    return data

solved_data = data
solve(solved_data, words)

('collections', (10, 15), True, ('collections',))
('subprocess', (12, 13), True, ('subprocess',))
('threading', (12, 7), True, ('threading',))
('datetime', (9, 5), True, ('datetime',))
('warnings', (7, 18), False, ('warnings',))
('tempfile', (7, 7), False, ('tempfile',))
('logging', (12, 11), False, ('logging',))
('h*s*lib', (5, 13), False, ('hashlib',))
('socket', (1, 19), False, ('socket',))
('json', (12, 10), True, ('json',))
('math', (10, 2), True, ('math',))
('sys', (7, 9), True, ('sys',))
('csv', (5, 17), False, ('csv',))
('re', (1, 18), True, ('re',))
('os', (10, 10), False, ('os',))
('collections', (10, 15), True, ('collections',))
('subprocess', (12, 13), True, ('subprocess',))
('threading', (12, 7), True, ('threading',))
('datetime', (9, 5), True, ('datetime',))
('warnings', (7, 18), False, ('warnings',))
('tempfile', (7, 7), False, ('tempfile',))
('logging', (12, 11), False, ('logging',))
('hashlib', (5, 13), False, ('hashlib',))
('socket', (1, 19), False, ('socket',))
('jso

['               c  r ',
 '              socket',
 '     d         l    ',
 '     a       s l    ',
 '     t t     u e    ',
 '     e hashlib csv  ',
 '     t r y   p t    ',
 'tempfile s warnings ',
 '  a  m a     o o    ',
 '  t  e d  j  c n    ',
 '  h    i os  e s    ',
 '       n  o  s      ',
 '     logging s      ']

In [264]:
res = requests.put(url, cookies=cookie, data='\n'.join(data))

In [10]:
lines.append('cprofile')
lines.append('concurrent')
lines.append('pkg_resources')
lines.append('encodings')
lines.append('antigravity')
lines.append('pip')
lines.append('xmlrpc')
lines.append('opcode')


In [11]:
lines = list(set(lines))

In [19]:
import requests
def crossword():
    url = 'https://crossword.stxnext.com/api/pyconpl2017/challenges'
    cookie = {'session': '.eJwlj0lqAzEQAP-isw-tXqSWPzNMb8QYEpixTyF_90DORUHVb9vqyPOr3V_HO29te0S7t1FiRjAFigMwxz6B9hjpZnkBscBCVmBfQlJCNPYC9MEGq0om7ivWgBxMac7Wg4jBTSBZfbiGghHbpLCe2QEgFKWvmVnebs3Po7bXzzO_r55Swd2C5nKZKyYTc2eokbUwUBXEEXVd3vvM43-Ctf19AAZ0PuY.DHh1Rg.tEB3jT39VrjOlBSf91s_pxlxIcI'}
    r = requests.get(url, cookies=cookie)
    urls = r.content.decode().split("\n")
    url = ([url for url in urls if len(url) > 5]).pop()
    ID = url.split('/').pop().replace('"', '')

   

    url = 'https://crossword.stxnext.com/api/pyconpl2017/challenges/{}'.format(ID)
    cookie = {'session': '.eJwlj0lqAzEQAP-isw-tXqSWPzNMb8QYEpixTyF_90DORUHVb9vqyPOr3V_HO29te0S7t1FiRjAFigMwxz6B9hjpZnkBscBCVmBfQlJCNPYC9MEGq0om7ivWgBxMac7Wg4jBTSBZfbiGghHbpLCe2QEgFKWvmVnebs3Po7bXzzO_r55Swd2C5nKZKyYTc2eokbUwUBXEEXVd3vvM43-Ctf19AAZ0PuY.DHh1Rg.tEB3jT39VrjOlBSf91s_pxlxIcI'}
    r = requests.get(url, cookies=cookie)

    from scrapy.selector import Selector
    url_base = 'https://crossword.stxnext.com/pyconpl2017/challenges/{}'.format(ID)
    req = requests.get(url_base, cookies=cookie)
    hints = Selector(text=req.content.decode()).xpath('/html/body/section/div/div/div[1]/pre/code/text()')
    all_hints = lines
    if hints:
        hints = hints.extract()[0].split('\n')
        hints = [hint.replace('django-', '').replace('django', '').lower() for hint in hints]
        hints.append('gridfs')
        hints.append('dateutil')
        hints.append('mockredis')
        all_hints += hints
        all_hints = list(set(all_hints))


    data = r.content.decode().split("\n")

    def get_words(data, down=True):
        words = []
        word = ''
        rows = len(data)
        columns = len(data[0])


        if down:
            for i in range(0, columns):
                for j in range(0, rows):
                    if data[j][i] != ' ':
                        word += data[j][i]
                    elif data[j][i] == ' ':
                        if word:
                            if len(word) > 1:
                                words.append((word, (j-1, i), down))
                        word = ''
                if word:
                    if len(word) > 1:
                        words.append((word, (j, i), down))
                    word = ''
        else:
            for j in range(0, rows):
                for i in range(0, columns):
                    if data[j][i] != ' ':
                        word += data[j][i]
                    elif data[j][i] == ' ':
                        if word:
                            if len(word) > 1:
                                words.append((word, (j, i-1), down))
                        word = ''
                if word:
                    if len(word) > 1:
                        words.append((word, (j, i), down))
                    word = ''
        return words

    words = get_words(data, False)
    words += get_words(data, True)

    import re
    def find_matches(words):
        result = []
        words.sort(key = lambda s: len(s[0]))
        words.reverse()
        for word in words:
            matches = []
            for line in all_hints:
                if len(line) == len(word[0]):
                    match = re.match(word[0].replace('*', '.'), line)
                    if match:
                        matches.append(match[0])
            if not matches:
                print('no match for {}'.format(word[0]))
            word = (word[0], word[1], word[2], tuple(matches))
            #print (word)
            result.append(word)
        return result

    words = find_matches(words)

    def data_fill(data, word):
        import random
        j, i = word[1]
        down = word[2]
        match = word[3][random.randint(0, len(word[3])-1)]
        length = len(match)

        if down:
            j -= length - 1
            x = 0
            for j in range(j, j+length):
                data[j] = list(data[j])
                data[j][i] = match[x]
                data[j] = ''.join(data[j])
                x += 1
        else:
            i -= length - 1
            x = 0

            data[j] = list(data[j])
            for i in range(i, i+length):
                data[j][i] = match[x]
                x += 1
            data[j] = ''.join(data[j])
        return data

    def solve(data, words, rerun=False):
        to_try = []
        for word in words:
            if len(word[3]) == 1 or (len(word[3]) > 1 and rerun):
                data = data_fill(data, word)
        words = get_words(data, False) +  get_words(data, True)
        words = find_matches(words)
        if not rerun:
            data = solve(data, words, True)
        #print(*data, sep='\n')
        return data

    solved_data = data
    print(*solve(solved_data, words), sep='\n')
    res = requests.put(url, cookies=cookie, data='\n'.join(data))
    return res.content

In [310]:
res = requests.put(url, cookies=cookie, data='\n'.join(data))

In [311]:
res.content
import 

b''

In [13]:
crossword()

                       s                                     
                       y                                     
                       s                                     
                       compileall                            
                       o                                     
                   c   n                                     
                 weakref                                     
                   l   i                                     
                 d e   g                                     
                 o n                                         
                 codeop                                      
             h t t a                                         
             tty e r                                c        
             m p s   a                    p      f  o        
           collections                    i   telnetlib  s   
               s     y                 f  p      m  o u  h   
        

b''

In [22]:
from requests.exceptions import ConnectionError
hello = False
def run_x(x):
    for i in range(0, x):
        crossword()
    return True
while not hello:
    try:
        hello=run_x(1000)
    except ConnectionError:
        pass



                                            z              
                                            i              
                                            p s            
                                     r    heapq            
                                  pipes     p l            
                                     a      p i            
                                  l  difflib  t b         m
                               decimal      atexit        a
                                  n  i        3 n z q     i
                                  e  n       c  a l u     l
                              asyncore       p wsgiref    c
                                  a          r  c b u     a
                                  c  h   sysconfig textwrap
                                d h  t  f    f  i      a   
                            unicodedata uu   i         r   
                          e     c    p  n  t l         n   
                        c m  q  t t     

                                            z              
                                            i              
                                            p s            
                                     r    heapq            
                                  pipes     p l            
                                     a      p i            
                                  l  difflib  t b         m
                               decimal      atexit        a
                                  n  i        3 n z q     i
                                  e  n       c  a l u     l
                              asyncore       p wsgiref    c
                                  a          r  c b u     a
                                  c  h   sysconfig textwrap
                                d h  t  f    f  i      a   
                            unicodedata uu   i         r   
                          e     c    p  n  t l         n   
                        c m  q  t t     

                                            z              
                                            i              
                                            p s            
                                     r    heapq            
                                  pipes     p l            
                                     a      p i            
                                  l  difflib  t b         m
                               decimal      atexit        a
                                  n  i        3 n z q     i
                                  e  n       c  a l u     l
                              asyncore       p wsgiref    c
                                  a          r  c b u     a
                                  c  h   sysconfig textwrap
                                d h  t  f    f  i      a   
                            unicodedata uu   i         r   
                          e     c    p  n  t l         n   
                        c m  q  t t     

                                            z              
                                            i              
                                            p s            
                                     r    heapq            
                                  pipes     p l            
                                     a      p i            
                                  l  difflib  t b         m
                               decimal      atexit        a
                                  n  i        3 n z q     i
                                  e  n       c  a l u     l
                              asyncore       p wsgiref    c
                                  a          r  c b u     a
                                  c  h   sysconfig textwrap
                                d h  t  f    f  i      a   
                            unicodedata uu   i         r   
                          e     c    p  n  t l         n   
                        c m  q  t t     

                                         c                           
                                    o  chunk                         
                                    s    r   g                       
                                    s  p struct                      
                                   tarfile                           
                                    u  p s                           
                                  pydoc                      c       
                                    i                      h o       
                    m           g  codeop                  t m       
                  a a      opcode d d                    s t p       
       e          s i           t e e                   quopri       
    json       copy l         u t csv                    b l l       
       c          n c s       r e i    p                 p i e       
       o          cmath d   shlex m    i             g   r b a       
     codecs     o o 

...

                                      l a                              
                                     io i                              
                                    i c f                              
                                   fnmatch                             
                                    s l                                
                               n    p e f                              
                         d   p i  g e   u j                            
                       itertools  encodings                            
                         c   p    t t   c o                            
                       gzip  l c  t     t n                            
                         m   i o  e syslog                             
                        math binhex   t o   f i                        
                         l     t  t   a l   c m                        
                          wsgiref  c  t stringprep   s          

                                      l a                              
                                     io i                              
                                    i c f                              
                                   fnmatch                             
                                    s l                                
                               n    p e f                              
                         d   p i  g e   u j                            
                       itertools  encodings                            
                         c   p    t t   c o                            
                       gzip  l c  t     t n                            
                         m   i o  e syslog                             
                        math binhex   t o   f i                        
                         l     t  t   a l   c m                        
                          wsgiref  c  t stringprep   s          

                                      l a                              
                                     io i                              
                                    i c f                              
                                   fnmatch                             
                                    s l                                
                               n    p e f                              
                         d   p i  g e   u j                            
                       itertools  encodings                            
                         c   p    t t   c o                            
                       gzip  l c  t     t n                            
                         m   i o  e syslog                             
                        math binhex   t o   f i                        
                         l     t  t   a l   c m                        
                          wsgiref  c  t stringprep   s          

                                      l a                              
                                     io i                              
                                    i c f                              
                                   fnmatch                             
                                    s l                                
                               n    p e f                              
                         d   p i  g e   u j                            
                       itertools  encodings                            
                         c   p    t t   c o                            
                       gzip  l c  t     t n                            
                         m   i o  e syslog                             
                        math binhex   t o   f i                        
                         l     t  t   a l   c m                        
                          wsgiref  c  t stringprep   s          

KeyboardInterrupt: 

In [439]:
**p*ib
import bdb

SyntaxError: invalid syntax (<ipython-input-439-c713106d8d31>, line 1)

In [1]:
lines

NameError: name 'lines' is not defined

In [ ]:
    url = 'https://crossword.stxnext.com/api/pyconpl2017/challenges'
    cookie = {'session': '.eJwlj0lqAzEQAP-isw-tXqSWPzNMb8QYEpixTyF_90DORUHVb9vqyPOr3V_HO29te0S7t1FiRjAFigMwxz6B9hjpZnkBscBCVmBfQlJCNPYC9MEGq0om7ivWgBxMac7Wg4jBTSBZfbiGghHbpLCe2QEgFKWvmVnebs3Po7bXzzO_r55Swd2C5nKZKyYTc2eokbUwUBXEEXVd3vvM43-Ctf19AAZ0PuY.DHh1Rg.tEB3jT39VrjOlBSf91s_pxlxIcI'}
    r = requests.get(url, cookies=cookie)
    urls = r.content.decode().split("\n")
    url = ([url for url in urls if len(url) > 5]).pop()
    ID = url.split('/').pop().replace('"', '')

   

    url = 'https://crossword.stxnext.com/api/pyconpl2017/challenges/{}'.format(ID)
    cookie = {'session': '.eJwlj0lqAzEQAP-isw-tXqSWPzNMb8QYEpixTyF_90DORUHVb9vqyPOr3V_HO29te0S7t1FiRjAFigMwxz6B9hjpZnkBscBCVmBfQlJCNPYC9MEGq0om7ivWgBxMac7Wg4jBTSBZfbiGghHbpLCe2QEgFKWvmVnebs3Po7bXzzO_r55Swd2C5nKZKyYTc2eokbUwUBXEEXVd3vvM43-Ctf19AAZ0PuY.DHh1Rg.tEB3jT39VrjOlBSf91s_pxlxIcI'}
    r = requests.get(url, cookies=cookie)

    from scrapy.selector import Selector
    url_base = 'https://crossword.stxnext.com/pyconpl2017/challenges/{}'.format(ID)
    req = requests.get(url_base, cookies=cookie)
    hints = Selector(text=req.content.decode()).xpath('/html/body/section/div/div/div[1]/pre/code/text()')
    all_hints = lines
    if hints:
        hints = hints.extract()[0].split('\n')
        hints = [hint.replace('django-', '').replace('django', '').lower() for hint in hints]
        hints.append('gridfs')
        hints.append('dateutil')
        hints.append('mockredis')
        all_hints += hints
        all_hints = list(set(all_hints))


    data = r.content.decode().split("\n")

In [26]:
initial_x=65 
initial_y=23

def getxy(x, y, initial_x, initial_y):
    xdiff = x-initial_x
    ydiff = y-initial_y
    if xdiff == ydiff:
        x += 2
        y += 4
    elif xdiff+2 == ydiff:
        x += 2
        y -=2
    elif xdiff-2 == ydiff:
        x -= 2
    return x,y
x, y = getxy(initial_x, initial_y, initial_x, initial_y)
print (x,y)
x, y = getxy(x, y, initial_x, initial_y)
print (x,y)
x, y = getxy(x, y, initial_x, initial_y)
print (x,y)
x, y = getxy(x, y, initial_x, initial_y)
print (x,y)
x, y = getxy(x, y, initial_x, initial_y)
print (x,y)
x, y = getxy(x, y, initial_x, initial_y)
print (x,y)
x, y = getxy(x, y, initial_x, initial_y)
print (x,y)
x, y = getxy(x, y, initial_x, initial_y)
print (x,y)
x, y = getxy(x, y, initial_x, initial_y)
print (x,y)


67 27
69 25
67 25
69 29
71 27
69 27
71 31
73 29
71 29


In [28]:
def getxy(x, y):
    if x == y:
        x += 2
        y += 4
    elif x+2 == y:
        x += 2
        y -=2
    elif x-2 == y:
        x -= 2
    return x,y
x, y = getxy(2, 4)
print (x,y)
x, y = getxy(x, y)
print (x,y)
x, y = getxy(x, y)
print (x,y)
x, y = getxy(x, y)
print (x,y)
x, y = getxy(x, y)
print (x,y)
x, y = getxy(x, y)
print (x,y)


4 2
2 2
4 6
6 4
4 4
6 8
